# Lesson 7 Class Exercises: Matplotlib

With these class exercises we learn a few new things.  When new knowledge is introduced you'll see the icon shown on the right: 
<span style="float:right; margin-left:10px; clear:both;">![Task](../media/new_knowledge.png)</span>

## Get Started
Import the Numpy, Pandas, Matplotlib packages and the Jupyter notebook Matplotlib magic

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Exercise 1. Load and clean the data for plotting

Import the Real Minimum Wages dataset from https://raw.githubusercontent.com/QuantEcon/lecture-source-py/master/source/_static/lecture_specific/pandas_panel/realwage.csv

In [23]:
wages=pd.read_csv('https://raw.githubusercontent.com/QuantEcon/lecture-source-py/master/source/_static/lecture_specific/pandas_panel/realwage.csv',
                  delimiter=',')
wages.head()

,Unnamed: 0,Time,Country,Series,Pay period,value
0,0,2006-01-01,Ireland,In 2015 constant prices at 2015 USD PPPs,Annual,17132.443
1,1,2007-01-01,Ireland,In 2015 constant prices at 2015 USD PPPs,Annual,18100.918
2,2,2008-01-01,Ireland,In 2015 constant prices at 2015 USD PPPs,Annual,17747.406
3,3,2009-01-01,Ireland,In 2015 constant prices at 2015 USD PPPs,Annual,18580.139
4,4,2010-01-01,Ireland,In 2015 constant prices at 2015 USD PPPs,Annual,18755.832


Clean the data by performing the following:
1. Add a new column containing just the year
2. Drop rows with missing values
3. Keep only rows in the series "In 2015 constant prices at 2015 USD PPPs"
4. Keep only rows where the pay period is 'Annual'
5. Drop unwanted columns: 'Unnamed: 0', 'Time' and 'Series'
6. Rename the 'value' column as 'Salary'
7. Reset the indexes

In [24]:
wages['Year'] = pd.to_datetime(wages['Time']).dt.year


In [25]:
wages.dropna(inplace=True)

In [26]:
wages = wages[(wages['Series'] == 'In 2015 constant prices at 2015 USD PPPs') & (wages['Pay period'] == 'Annual')]

In [27]:
wages['Series'].unique()

array(['In 2015 constant prices at 2015 USD PPPs'], dtype=object)

In [28]:
wages.drop(['Unnamed: 0', 'Time', 'Series'], axis=1, inplace=True)


In [30]:
wages.rename(columns={'value': 'Salary'}, inplace=True)
wages

,Country,Pay period,Salary,Year
0,Ireland,Annual,17132.4430,2006
1,Ireland,Annual,18100.9180,2007
2,Ireland,Annual,17747.4060,2008
3,Ireland,Annual,18580.1390,2009
4,Ireland,Annual,18755.8320,2010
...,...,...,...,...
1329,Colombia,Annual,6983.5298,2015
1330,Colombia,Annual,6950.1768,2016
1372,Costa Rica,Annual,9967.0381,2014
1373,Costa Rica,Annual,10561.5360,2015


In [35]:
wages['Salary'] = wages['Salary'].astype('float')

In [40]:
wages.reset_index(inplace=True, drop=True)

In [41]:
wages

,Country,Pay period,Salary,Year
0,Ireland,Annual,17132.4430,2006
1,Ireland,Annual,18100.9180,2007
2,Ireland,Annual,17747.4060,2008
3,Ireland,Annual,18580.1390,2009
4,Ireland,Annual,18755.8320,2010
...,...,...,...,...
330,Colombia,Annual,6983.5298,2015
331,Colombia,Annual,6950.1768,2016
332,Costa Rica,Annual,9967.0381,2014
333,Costa Rica,Annual,10561.5360,2015


## Exercise 2. Add a quartile group column

Find the quartile to which each row of the `Minimum_Salary` column belongs.  Use the [pd.DataFrame.quantile()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html) function to to find the exact quartiles.  Add a new column to the dataframe named `Group` that contains the values `QG1`, `QG2`, `QG3` or `QG4` representeding the quartile group (QG) to which the row belongs. Rows with a value between 0 and the first quartile get the value QG1, rows between the 1st and 2nd quartile get the value QG2, etc.  The resulting table should look like the following:

| Country | Pay period | Minimum_Salary | Year | Group | 
| -- | -- | -- | -- | -- |
| Turkey | Annual | 9379.5137| 2014 | QG2 | 
| Israel | Annual | 11428.6940 | 2006 | QG2 | 
| Colombia | Annual | 6902.3120 | 2013 | QG1 | 
| Turkey | Annual | 8514.1846 | 2009 | QG2 | 
| Costa Rica | Annual | 10859.7470 | 2016 | QG2 | 

Step 1: Find the quartiles for teh `Minimum_Salary` column

In [42]:
q1 = wages['Salary'].quantile(0.25)
q2 = wages['Salary'].quantile(0.5)
q3 = wages['Salary'].quantile(0.75)
q4 = wages['Salary'].quantile(1)
print(q1, q2, q3, q4)

6952.0789 11442.349 16778.677 23401.492


Step 2: Create a new Series object of the correct size that will eventually house our Group categories.  

In [43]:
group = pd.Series(np.zeros(wages.shape[0]))
group

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
330    0.0
331    0.0
332    0.0
333    0.0
334    0.0
Length: 335, dtype: float64

Step 3: Set the quartile groups for series relative to what it should be in the rows of the dataframe, and then add that series to our dataframe.

In [44]:
group[(wages['Salary'] > 0) & (wages['Salary'] <= q1)]
group[(wages['Salary'] > q1) & (wages['Salary'] <= q2)]
group[(wages['Salary'] > q2) & (wages['Salary'] <= q3)]
group[(wages['Salary'] > q3) & (wages['Salary'] <= q4)]


0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
272    0.0
273    0.0
274    0.0
275    0.0
276    0.0
Length: 84, dtype: float64

## Exercise 3. Create a boxplot

Create a graph using a single axis that shows the boxplots of the four groups. You can use the [matplotlib.axes.Axes.boxplot()](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html) function to do this.  This will allow us to see if we properly separated rows by quartiles. It will also allow us to see the spread of the data in each quartile. Be sure to lable the x-axis tick marks with the proper quantile group name.

## Exercise 4. Create a Scatterplot

Create a single scatterplot to explore if the salaries in quartile group 1 and quartile group 4 are correlated.

Step 1: Transform the data such that each quantile group is a separate column with an index of `Year`.

Step 2: Now generate the plot

Recreate the plot above, but set a different color per year and size the points to be larger for later years and smaller for earlier years.

Step 1:  Create Numpy arrays to represent the colors and sizes for each point of data. 

Step 2: Create the figure

## Exercise 5. Create a grid of scatterplots

Now, let's see the pairwise scatterplot of each quartile group with every other group.  Create a 4x4 grid of subplots. The rows and columns of the subplot represent one of the 4 groups and each plot represents the scatterplot of those groups.  The plot should look similar to the following:

<img src="media/D08-multiplot.png" width="75%">


To ensure that the plot is not too small to read we can force a larger size of 15 inches by 15 inches with the following:

```python
plt.rcParams["figure.figsize"] = (15, 15)
```

Also, the x-axis and y-axis labels do not need to be repeated, we only need to set them on the first column and last rows.  You can set the y-axis labels on the first column by using the `set` function of the Axes function and providing the `ylabel` argument. For example.
```python
axes[0, 0].set(ylabel="QG1")
```

You can do the same for the x-axis on the bottom row using the same style:
```python
axes[3, 0].set(xlabel="QG1")
```

**Hint**: You will need to repeat a lot of code.  Cut-and-paste is your friend!  

In [ ]:
# Create the figure and 4 x 4 set of axes.

# Add the first row of scatterplots

# Add the second row of scatterplots

# Add the third row of scatterplots

# Add the fourth row of scatterplots

# Set the x-axis lables only on the left-most plots

# Set the y-axis labels only on the bottom plots.

# Show the plot


## Exercise 6. Create a grid of line plots
Now, let's create a line graph of changes over time for each quartile group.  Let's use a 2x2 subplot grid with each grid showing a different group.


<img src="media/D08-lineplot.png" width="50%">